In [1]:
!pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.6/589.6 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 54.9 MB

In [19]:
!pip install -q huggingface_hub

# Large Language Models

### LLMs

In [13]:
from langchain.llms import OpenAI
import os

In [14]:
os.environ['OPENAI_API_KEY'] = ''

In [15]:
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0.5)

In [16]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



Monday:
Warm up: 5 minutes of jogging
Main workout:
- 30 minutes of outdoor cycling
- 3 sets of 10 push-ups
- 3 sets of 10 squats
- 3 sets of 10 lunges
Cool down: 5 minutes of stretching

Tuesday:
Warm up: 5 minutes of jumping jacks
Main workout:
- 30 minutes of outdoor running
- 3 sets of 10 burpees
- 3 sets of 10 mountain climbers
- 3 sets of 10 Russian twists
Cool down: 5 minutes of yoga poses

Wednesday:
Active rest day:
- Go for a leisurely hike or walk in nature for 30 minutes
- Do some light stretching or yoga

Thursday:
Warm up: 5 minutes of high knees
Main workout:
- 30 minutes of outdoor swimming
- 3 sets of 10 tricep dips using a park bench
- 3 sets of 10 step-ups on a park bench
- 3 sets of 10 bicycle crunches
Cool down: 5 minutes of foam rolling

Friday:
Warm up: 5 minutes of skipping rope
Main workout:
- 30 minutes of outdoor HIIT (High-


### Few Shot Learning

In [18]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# load the model
chat = ChatOpenAI(model_name="gpt-4", temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the meaning of life?")

"The meaning of life? Well, I believe it's a jigsaw puzzle. You have to assemble it yourself and everyone's looks a little different. But remember, don't force a piece that doesn't fit!"

### Applications


Creating a Question-Answering Prompt Template

In [20]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

In [21]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [22]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


paris


In [39]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
print(llm_chain.run(questions=qs_str))

Paris
Blue whale
Nitrogen
Yellow


Text Summarization

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [29]:
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [34]:
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.run(text=text)

In [35]:
print(summarized_text)

LangChain offers various modules for building language model applications, which can be combined to create complex applications or used individually for simpler ones, with the most basic building block being calling an LLM on input, as demonstrated in a simple example of generating a company name based on its product.


use .run() when you have a more complex workflow or pipeline that involves multiple steps or components, and use .predict() when you have a specific, isolated model and want to perform a single inference or prediction.

Both methods are valid and will work as long as you have an LLMChain instance defined. The choice between the two depends on your specific use case and personal preference. If you need more control and flexibility, the .run() method may be more appropriate. If you're looking for a simpler, more streamlined interface, the .predict() method can be a better fit.



Text Translation

In [40]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [43]:
source_language = "English"
target_language = "Tagalog"
text = "The weather today is very hot and alarming to some places"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)

In [44]:
print(translated_text)

Ang panahon ngayon ay napakainit at nakababahala sa ilang lugar.
